# Project Description:

The purpose of this project is to implement a neural network that performs the translation of mathematical formulae from traditional **infix notation**—where the operator appears between two operands—to **postfix** (also known as Reverse Polish Notation), where the operator follows the operands.

Infix notation is the most commonly used in human-readable mathematics (e.g., a + b), but it is inherently ambiguous without additional syntactic aids such as parentheses or operator precedence rules. This ambiguity arises because different parse trees can correspond to the same expression depending on how operations are grouped.

In contrast, postfix notation eliminates the need for parentheses entirely. The order of operations is explicitly encoded by the position of the operators relative to the operands, making it more suitable for stack-based evaluation and easier to parse programmatically.

**Example:**

Consider the ambiguous infix expression:
a + b * c

This expression can be parsed in at least two different ways:

Interpretation (Infix):	(a + b) * c	   
Equivalent Postfix: ab+c*

Interpretation (Infix):	a + (b * c)	          
Equivalent Postfix: abc*+


This project aims to learn such disambiguations and generate the correct postfix form from a given infix expression using a data-driven approach based on neural networks.

To simplify the task and control the complexity of expressions, we restrict our dataset to formulae with a maximum syntactic depth of 4. This means that the abstract syntax trees representing these expressions will have at most three levels, ensuring that the neural network operates on a bounded and manageable set of possible structures.

In [27]:
import numpy as np
import random
import string
import tensorflow as tf
from tensorflow.keras import layers, models

In [28]:
#Setting random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

We build formulae using 6 identifiers a,b,c,d,e,f and 4 binary operators +,-,*,/.
For simplicity we do not take advantage of precedence or associativity rules for infix notation, and suppose that all binary operations as always fully parenthesizes: (e1 op e2).

In [29]:
# -------------------- Constants --------------------
OPERATORS = ['+', '-', '*', '/']
IDENTIFIERS = list('abcdef')
SPECIAL_TOKENS = ['PAD', 'SOS', 'EOS']
SYMBOLS = ['(', ')', '+', '-', '*', '/']
VOCAB = SPECIAL_TOKENS + SYMBOLS + IDENTIFIERS + ['JUNK'] #may use junk in autoregressive generation

token_to_id = {tok: i for i, tok in enumerate(VOCAB)}
id_to_token = {i: tok for tok, i in token_to_id.items()}
VOCAB_SIZE = len(VOCAB)
PAD_ID = token_to_id['PAD']
EOS_ID = token_to_id['EOS']
SOS_ID = token_to_id['SOS']

MAX_DEPTH = 4
MAX_LEN = 4*2**MAX_DEPTH -2 #enough to fit expressions at given depth (+ EOS)

In [30]:
# -------------------- Expression Generation --------------------
def generate_infix_expression(max_depth):
    if max_depth == 0:
        return random.choice(IDENTIFIERS)
    elif random.random() < 0.25:
        return generate_infix_expression(max_depth - 1)
    else:
        left = generate_infix_expression(max_depth - 1)
        right = generate_infix_expression(max_depth - 1)
        op = random.choice(OPERATORS)
        return f'({left} {op} {right})'

def tokenize(expr):
    return [c for c in expr if c in token_to_id]

def infix_to_postfix(tokens):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    output, stack = [], []
    for token in tokens:
        if token in IDENTIFIERS:
            output.append(token)
        elif token in OPERATORS:
            while stack and stack[-1] in OPERATORS and precedence[stack[-1]] >= precedence[token]:
                output.append(stack.pop())
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()
    while stack:
        output.append(stack.pop())
    return output

def encode(tokens, max_len=MAX_LEN):
    ids = [token_to_id[t] for t in tokens] + [EOS_ID]
    return ids + [PAD_ID] * (max_len - len(ids))

def decode_sequence(token_ids, id_to_token, pad_token='PAD', eos_token='EOS'):
    """
    Converts a list of token IDs into a readable string by decoding tokens.
    Stops at the first EOS token if present, and ignores PAD tokens.
    """
    tokens = []
    for token_id in token_ids:
        token = id_to_token.get(token_id, '?')
        if token == eos_token:
            break
        if token != pad_token:
            tokens.append(token)
    return ' '.join(tokens)

def generate_dataset(n,max_depth=MAX_DEPTH):
    X, Y = [], []
    for _ in range(n):
        expr = generate_infix_expression(MAX_DEPTH)
        #expr = expr_gen.generate(max_depth=max_dthep)
        infix = tokenize(expr)
        postfix = infix_to_postfix(infix)
        X.append(encode(infix))
        Y.append(encode(postfix))
    return np.array(X), np.array(Y)

#you might use the shift function for teacher-forcing
def shift_right(seqs):
    shifted = np.zeros_like(seqs)
    shifted[:, 1:] = seqs[:, :-1]
    shifted[:, 0] = SOS_ID
    return shifted

Let us define a simple dataset, and inspect a few samples. You can use a larger dataset, if you want, or directly use the generator.

In [31]:
X_train, Y_train = generate_dataset(10000)
decoder_input_train = shift_right(Y_train)

# Dataset
X_val, Y_val = generate_dataset(1000)
decoder_input_val = shift_right(Y_val)

In [32]:
i =  np.random.randint(10000)
print(i)
print("infix : ",decode_sequence(X_train[i],id_to_token))
print("posfix notation: ",decode_sequence(Y_train[i],id_to_token))
print("teacher forcing : ", decode_sequence(decoder_input_train[i],id_to_token))

7270
infix :  ( ( ( d + f ) / ( ( e + c ) * ( c - e ) ) ) * ( ( ( c * e ) + ( c + d ) ) * ( ( a / d ) / ( f + a ) ) ) )
posfix notation:  d f + e c + c e - * / c e * c d + + a d / f a + / * *
teacher forcing :  SOS d f + e c + c e - * / c e * c d + + a d / f a + / * *


# Constraints
* You may use any architecture (decoder-only, encoder-decoder, or other).

* The maximum number of parameters is 2 million.

* Beam search is not allowed.

* You may adapt the formula generator to your needs, but preserve its core logic—especially the frequency distribution of formulas by depth, as it may significantly influence model performance.

* You may train your model using a pre-generated fixed dataset (e.g., an array) or directly use an on-the-fly generator.




In [35]:
# Model hyperparameters
EMBEDDING_DIM = 128
LSTM_UNITS = 256
DROPOUT_RATE = 0.2

# Encoder
encoder_inputs = layers.Input(shape=(None,), name='encoder_inputs')
encoder_embedding = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(encoder_inputs)
encoder_lstm = layers.LSTM(LSTM_UNITS, return_state=True, dropout=DROPOUT_RATE, name='encoder_lstm')
encoder_outputs, encoder_h, encoder_c = encoder_lstm(encoder_embedding)
encoder_states = [encoder_h, encoder_c]

# Decoder
decoder_inputs = layers.Input(shape=(None,), name='decoder_inputs')
decoder_embedding = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(decoder_inputs)
decoder_lstm = layers.LSTM(LSTM_UNITS, return_sequences=True, return_state=True,
                           dropout=DROPOUT_RATE, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = layers.Dense(VOCAB_SIZE, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Training model
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model summary
model.summary()

# Let's count parameters
total_params = model.count_params()
print(f"\nTotal parameters: {total_params:,}")
print(f"Parameter limit: 2,000,000")
print(f"Within limit: {total_params <= 2000000}")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 128) │      2,048 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │      2,048 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    394,240 │ embedding_2[0][0… │
│                     │ (None, 256),      │            │ not_equal_2[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    394,240 │ embedding_3[0][0… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 16)  │      4,112 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 796,688 (3.04 MB)

 Trainable params: 796,688 (3.04 MB)

 Non-trainable params: 0 (0.00 B)


Total parameters: 796,688
Parameter limit: 2,000,000
Within limit: True


In [36]:
#For autoregressive generation
encoder_model = models.Model(encoder_inputs, [encoder_h, encoder_c])

# Decoder model for inference
decoder_state_input_h = layers.Input(shape=(LSTM_UNITS,))
decoder_state_input_c = layers.Input(shape=(LSTM_UNITS,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inf, decoder_h_inf, decoder_c_inf = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states_inf = [decoder_h_inf, decoder_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf
)

def autoregressive_decode(model, encoder_input, max_len=MAX_LEN):
    """
    Autoregressively decode the postfix notation from infix input.

    This function implements AUTOREGRESSIVE generation:
    - Takes ONLY infix notation as input (encoded)
    - Generates postfix tokens ONE BY ONE (autoregressively)
    - Each token is generated based on previous tokens
    - Uses greedy decoding (no beam search)

    Args:
        model: The trained model (not used directly, but encoder_model and decoder_model are)
        encoder_input: Encoded infix sequence (numpy array)
        max_len: Maximum length of generated sequence

    Returns:
        Generated postfix sequence (numpy array of token IDs, starting with SOS)
    """
    encoder_input = np.array(encoder_input)
    encoder_input = encoder_input.reshape(1, -1)

    # Get encoder states from infix input
    encoder_h, encoder_c = encoder_model.predict(encoder_input, verbose=0)

    # Initialize decoder state with encoder states
    decoder_state = [encoder_h, encoder_c]

    # Start with SOS token
    decoder_input = np.array([[SOS_ID]])
    generated = [SOS_ID]

    # AUTOREGRESSIVE GENERATION
    for _ in range(max_len - 1):
        output_tokens, decoder_h, decoder_c = decoder_model.predict(
            [decoder_input] + decoder_state, verbose=0)

        # Get the predicted token (greedy decoding - no beam search:) )
        predicted_id = np.argmax(output_tokens[0, 0, :])
        generated.append(int(predicted_id))

        if predicted_id == EOS_ID:
            break
        decoder_state = [decoder_h, decoder_c]
        decoder_input = np.array([[predicted_id]])

    return np.array(generated)

In [38]:
#Model training
Y_train_target = Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1)
Y_val_target = Y_val.reshape(Y_val.shape[0], Y_val.shape[1], 1)

# Training configuration
BATCH_SIZE = 32
EPOCHS = 100
DROPOUT_RATE = 0.25

# Train the model
history = model.fit(
    [X_train, decoder_input_train],
    Y_train_target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=([X_val, decoder_input_val], Y_val_target),
    verbose=1
)

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.2775 - loss: 2.0400 - val_accuracy: 0.1799 - val_loss: 1.6333
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.1616 - loss: 1.6234 - val_accuracy: 0.1314 - val_loss: 1.5650
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.1398 - loss: 1.5230 - val_accuracy: 0.1672 - val_loss: 1.2980
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.1822 - loss: 1.2278 - val_accuracy: 0.2336 - val_loss: 0.8667
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.2484 - loss: 0.8279 - val_accuracy: 0.3047 - val_loss: 0.5836
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.3114 - loss: 0.5800 - val_accuracy: 0.3118 - val_loss: 0.4237
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.3292 - loss: 0.4259 - val_accuracy: 0.3837 - val_loss: 0.3678
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.3786 - loss: 0.3288 - 

# Evaluation

We shall evaluate a generated item y_pred using "prefix accuracy", the lenght of
the initial prefix of y_pred matching the ground true y_true. This will be divided by the maximum length of y_true and y_pred (up to EOS), so that a perfect match has score 1.

* It's more informative than exact match (which is often 0)

* It’s tighter than edit distance: focuses on generation flow

* Captures where the model starts to make errors



In [33]:
def prefix_accuracy_single(y_true, y_pred, id_to_token, eos_id=EOS_ID, verbose=False):
    t_str = decode_sequence(y_true, id_to_token).split(' EOS')[0]
    p_str = decode_sequence(y_pred, id_to_token).split(' EOS')[0]
    t_tokens = t_str.strip().split()
    p_tokens = p_str.strip().split()
    print(len(p_tokens))
    max_len = max(len(t_tokens), len(p_tokens))
    n = min(len(t_tokens), len(p_tokens))
    match_len = 0
    while match_len < n and t_tokens[match_len] == p_tokens[match_len]:
        match_len += 1
    score = match_len / max_len if max_len>0 else 0
    if verbose:
        print("TARGET :", ' '.join(t_tokens))
        print("PREDICT:", ' '.join(p_tokens))
        print(f"PREFIX MATCH: {match_len}/{len(t_tokens)} → {score:.2f}")

    return score

In [34]:
#example
i = np.random.randint(10000)
expected = np.copy(Y_train[i])
generated = np.copy(Y_train[i])
expr_len = np.where(generated==EOS_ID)[0][0]
j = np.random.randint(expr_len)
generated[j] = EOS_ID #making shorter
prefix_accuracy_single(expected,generated,id_to_token,verbose=True)

14
TARGET : a e / b a - / e f / d d + + - c b + b - c e + b / / -
PREDICT: a e / b a - / e f / d d + +
PREFIX MATCH: 14/27 → 0.52


0.5185185185185185

For the exam, evaluate you model on a test set of 30 expressions. Repeat this evaluation 10 times, and return the mean and std for this rounds.

In [39]:
def test(no=30,rounds=10):
  rscores =[]
  for i in range(rounds):
    print("round=",i)
    X_test, Y_test = generate_dataset(no)
    scores = []
    for j in range(no):
      encoder_input=X_test[j]
      generated = autoregressive_decode(model, encoder_input)[1:] #remove SOS
      scores.append(prefix_accuracy_single(Y_test[j], generated, id_to_token))
    rscores.append(np.mean(scores))
  return np.mean(rscores),np.std(rscores)

res, std = test(30,10)
print("score=",res,"std=",std)

round= 0
21
23
13
5
27
19
21
15
17
19
13
27
13
27
23
17
29
23
21
15
23
27
23
19
13
19
17
15
19
11
round= 1
19
15
15
19
25
27
15
13
19
23
25
15
27
7
3
31
15
29
27
13
21
7
3
13
25
27
3
9
23
13
round= 2
29
7
9
11
11
13
7
23
15
27
23
25
7
21
11
27
15
15
17
11
15
23
13
23
21
27
13
13
15
23
round= 3
19
11
21
3
21
9
5
9
11
23
13
15
11
13
11
15
15
17
19
15
21
11
27
7
17
17
15
23
25
27
round= 4
9
9
25
11
27
13
25
29
19
7
29
23
23
19
5
15
19
9
13
15
9
15
21
11
17
15
23
19
7
9
round= 5
25
9
23
13
9
23
3
29
23
15
7
19
17
13
23
15
15
9
13
23
21
15
3
13
23
19
17
27
27
9
round= 6
15
23
27
23
17
5
21
27
5
21
23
15
7
13
1
5
19
29
17
13
15
11
11
21
17
15
11
15
5
9
round= 7
15
9
25
25
23
13
7
7
17
21
9
27
21
25
11
23
15
23
21
19
13
23
27
23
21
15
17
25
5
23
round= 8
3
15
27
7
19
23
15
25
11
19
27
11
13
21
19
15
9
13
7
9
19
19
11
21
27
25
27
21
19
21
round= 9
13
7
11
11
19
9
9
25
13
15
19
23
3
3
19
11
31
11
9
17
13
3
19
13
25
11
15
7
19
17
score= 0.995351724137931 std= 0.0059665496415806105


In [43]:
model.save_weights('model_weights.weights.h5')
print("Model weights saved to 'model_weights.weights.h5'")

Model weights saved to 'model_weights.weights.h5'


In [45]:
File_id = "1hclDt8Kzo0b69EYGuIsuL_ffZptk9KK7"

In [42]:
!gdown --id 1hclDt8Kzo0b69EYGuIsuL_ffZptk9KK7

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hclDt8Kzo0b69EYGuIsuL_ffZptk9KK7
To: /content/model_weights.weights.h5
100% 9.60M/9.60M [00:00<00:00, 14.4MB/s]


Be sure to evalutate the generator: your model may only take as input the expression in infix format and return its translation to postifix.

If you are usuing an encoder-decoder model, generation must be done autoregressively.

# What to deliver

As usual you are supposed to deliver a single notebook witten in Keras. You are auhtorized to use Keras3 with pytorch as backend if you prefer.

Do no upload a zip file: the submission will be rejected.

The python notebook should have a clear documentation of the training phase, possibly with its history.

Please provide the network parameters by means of gdown.